In [8]:
# 필요한 라이브러리들을 로드하자
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc

In [22]:
# 인베스팅닷컴 열기
driver = uc.Chrome()
url = 'https://kr.investing.com/economic-calendar/cpi-733'  # 원하는 웹사이트 주소로 변경하세요
driver.get(url)

actions = ActionChains(driver)

In [23]:
# '더 보여주기' 버튼을 몇 번 클릭할건지 지정
for i in range(80):
    element = driver.find_element(By.CSS_SELECTOR, '.showMoreReplies.block')
    actions.click(element).perform() 

In [24]:
### 인베스팅닷컴에 나온 정보를 크롤링

html = driver.page_source
# BeautifulSoup 객체를 생성
soup = bs(html, 'html.parser')
# 클래스가 "chart-wrapper"인 요소를 찾아서 해당하는 것을 data에 저장
wrapper = soup.find('table', attrs={'class': 'genTbl openTbl ecHistoryTbl'})
data = wrapper.prettify()

In [72]:
# 날짜들을 뽑아오자
dates = []
for td in wrapper.find_all('td', {'class': 'left'}):
    date = td.get_text(strip=True)  # 공백 제거
    # date = date.replace("'", '')  # 따옴표 제거
    # date = date.strip()  # 추가적인 공백 제거
    dates.append(date)

dates_droped = dates[::2]
# even_index_elements = your_list[::2]
# odd_index_elements = your_list[1::2]

In [73]:
# 날짜에 대응하는 값들을 뽑아오자
values = []
for span in wrapper.find_all('span', {'class': ['redFont', 'blackFont', 'greenFont']}):
    value = span.get_text(strip=True)  # 공백 제거
    values.append(value)

In [76]:
# 뽑아온 날짜와 숫자 데이터를 활용하여 데이터 프레임을 생성하자
df = pd.DataFrame({
    'date': dates_droped,
    'uscpi_yoy': values
})

In [78]:
# 인베스팅 닷컴에서 크롤링해온 데이터를 csv파일 형태로 저장하자
df.to_csv('uscpi_yoy_lag.csv', encoding='cp949', index=False)

In [79]:
pd.read_csv('uscpi_yoy_lag.csv', encoding='cp949')

,date,uscpi_yoy
0,2023년 06월 13일 (5월),4.0%
1,2023년 05월 10일 (4월),4.9%
2,2023년 04월 12일 (3월),5.0%
3,2023년 03월 14일 (2월),6.0%
4,2023년 02월 14일 (1월),6.4%
...,...,...
253,2002년 05월 01일 (4월),1.60%
254,2002년 04월 01일 (3월),1.50%
255,2002년 03월 01일 (2월),1.10%
256,2002년 02월 01일 (1월),1.10%


In [ ]:
# # 데이터에서 값 추출
# c1 = '<span class="redFont" title="예상치보다 낮습니다">'
# c2 = '<span class="blackFont" title="기대치에 따라">'
# c3 = '<span class="greenFont" title="예상치보다 높습니다">'

# if data.find(c1) != -1:
#     number_start_index = data.find(c1) + len(c1)
#     number_end_index = data.find("</span>", number_start_index)
#     number = data[number_start_index:number_end_index].strip() 
# elif data.find(c2) != -1:
#     number_start_index = data.find(c2) + len(c2)
#     number_end_index = data.find("</span>", number_start_index)
#     number = data[number_start_index:number_end_index].strip()  
# elif data.find(c3) != -1:
#     number_start_index = data.find(c3) + len(c3)
#     number_end_index = data.find("</span>", number_start_index)
#     number = data[number_start_index:number_end_index].strip()  

# print(number)